In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
#Import Excel data
stock_data = pd.read_excel('/content/drive/MyDrive/IEOR4574:Project_1/online_retail_II.xlsx', sheet_name = ['Year 2009-2010','Year 2010-2011'])

In [ ]:
stock_data_1 = stock_data.get('Year 2009-2010')
stock_data_2 = stock_data.get('Year 2010-2011')

In [ ]:
#Assess sheet1's data quality
stock_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [ ]:
#Closer look at Description
a = stock_data_1[(stock_data_1['Description'].isnull())]

In [ ]:
a.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.0,NaN,United Kingdom
3114,489655,20683,NaN,-44,2009-12-01 17:26:00,0.0,NaN,United Kingdom
3161,489659,21350,NaN,230,2009-12-01 17:39:00,0.0,NaN,United Kingdom
3731,489781,84292,NaN,17,2009-12-02 11:45:00,0.0,NaN,United Kingdom
4296,489806,18010,NaN,-770,2009-12-02 12:42:00,0.0,NaN,United Kingdom


In [ ]:
a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2928 entries, 470 to 525233
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      2928 non-null   object        
 1   StockCode    2928 non-null   object        
 2   Description  0 non-null      object        
 3   Quantity     2928 non-null   int64         
 4   InvoiceDate  2928 non-null   datetime64[ns]
 5   Price        2928 non-null   float64       
 6   Customer ID  0 non-null      float64       
 7   Country      2928 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 205.9+ KB


In [ ]:
a.Price.describe()

count    2928.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: Price, dtype: float64

In [ ]:
#Since Price of null descrition in the records are all zero, all these records are meaningless and removable
stock_data_1 = stock_data_1[(stock_data_1['Description'].notnull())]
stock_data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522533 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      522533 non-null  object        
 1   StockCode    522533 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     522533 non-null  int64         
 4   InvoiceDate  522533 non-null  datetime64[ns]
 5   Price        522533 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      522533 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 35.9+ MB


In [ ]:
#Closer look at Invoice
invoice = stock_data_1.Invoice.astype('string')

In [ ]:
import re
test = [re.fullmatch("\w+\d{5}?",i) for i in invoice]
test1 = [re.fullmatch("\d{6}",i) for i in invoice]
test2 =  [re.fullmatch("\AC\d{6}",i) for i in invoice]
test3 = [re.fullmatch("\w\d{6}",i) for i in invoice]
index = []
for i in range(len(test)):
    if test2[i] is None:
        if test3[i] is not None:
            index.append(i)
print(index)

[178089, 274230, 401031]


In [ ]:
stock_data_1.iloc[index]
#Since bad debt are needed in bookkeeping, thus we retain them. 

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,-53594.36,NaN,United Kingdom
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,-44031.79,NaN,United Kingdom
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,-38925.87,NaN,United Kingdom


In [ ]:
#Closer look at Description again, tease out nonsense records
description = stock_data_1.Description.astype('string')

In [ ]:
description.value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    3549
REGENCY CAKESTAND 3 TIER              2212
STRAWBERRY CERAMIC TRINKET BOX        1843
PACK OF 72 RETRO SPOT CAKE CASES      1466
ASSORTED COLOUR BIRD ORNAMENT         1457
                                      ... 
RED MINI TAPE MEASURE                    1
*USB Office Mirror Ball                  1
Sold as C                                1
PALE PINK/AMETHYST STONE NECKLACE        1
dotcom email                             1
Name: Description, Length: 4681, dtype: Int64

In [ ]:
test11 = [re.match("^\?",i) for i in description]
test12 = [re.fullmatch("\d{5}",i) for i in description]
index2 = [i for i in range(len(test11)) if test11[i] != None] + [i for i in range(len(test12)) if test12[i] != None]

In [ ]:
index2.sort()

In [ ]:
index3 = stock_data_1.iloc[index2].index

In [ ]:
index3

Int64Index([  6911,  63439, 100428, 167838, 171218, 183682, 193991, 194006,
            194012, 228090, 238366, 238394, 242204, 249029, 249032, 256280,
            262387, 272375, 272461, 272476, 272477, 274052, 274053, 303575,
            303627, 307123, 307127, 309842, 320313, 331159, 331505, 336564,
            340002, 342265, 350797, 350848, 364012, 374512, 398695, 400017,
            427129, 428975, 429085, 431432, 434346, 434372, 451858, 481383,
            510251, 524457],
           dtype='int64')

In [ ]:
#Get rid of nonsense records 
stock_data_1.drop(index3, inplace = True)

In [ ]:
stock_data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522483 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      522483 non-null  object        
 1   StockCode    522483 non-null  object        
 2   Description  522483 non-null  object        
 3   Quantity     522483 non-null  int64         
 4   InvoiceDate  522483 non-null  datetime64[ns]
 5   Price        522483 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      522483 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 35.9+ MB


In [ ]:
#Finally check if there is abnormal data in Price section
stock_data_1.Price.describe()

count    522483.000000
mean          4.715559
std         146.542333
min      -53594.360000
25%           1.250000
50%           2.100000
75%           4.210000
max       25111.090000
Name: Price, dtype: float64

In [ ]:
#Because price = 0 has no impact on statistics for sales, we could delete all records with unit price = 0.
stock_data_1 = stock_data_1[(stock_data_1.Price != 0)]

In [ ]:
stock_data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 521774 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      521774 non-null  object        
 1   StockCode    521774 non-null  object        
 2   Description  521774 non-null  object        
 3   Quantity     521774 non-null  int64         
 4   InvoiceDate  521774 non-null  datetime64[ns]
 5   Price        521774 non-null  float64       
 6   Customer ID  417503 non-null  float64       
 7   Country      521774 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 35.8+ MB


In [ ]:
stock_data_1[(stock_data_1.Price == 0)]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country


In [ ]:
#Follow the same logic, processing stock_data_2
stock_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [ ]:
stock_data_2 = stock_data_2[(stock_data_2['Description'].notnull())]
stock_data_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 540456 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      540456 non-null  object        
 1   StockCode    540456 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     540456 non-null  int64         
 4   InvoiceDate  540456 non-null  datetime64[ns]
 5   Price        540456 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      540456 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 37.1+ MB


In [ ]:
description = stock_data_2.Description.astype('string')
test11 = [re.match("^\?",i) for i in description]
test12 = [re.fullmatch("\d{5}",i) for i in description]
index2 = [i for i in range(len(test11)) if test11[i] != None] + [i for i in range(len(test12)) if test12[i] != None]
index2.sort()
index3 = stock_data_2.iloc[index2].index
stock_data_2.drop(index3, inplace = True)

In [ ]:
stock_data_2 = stock_data_2[(stock_data_2.Price != 0)]

In [ ]:
stock_data_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 539395 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      539395 non-null  object        
 1   StockCode    539395 non-null  object        
 2   Description  539395 non-null  object        
 3   Quantity     539395 non-null  int64         
 4   InvoiceDate  539395 non-null  datetime64[ns]
 5   Price        539395 non-null  float64       
 6   Customer ID  406790 non-null  float64       
 7   Country      539395 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 37.0+ MB


In [ ]:
#concatenate two sheet into one
stock_data = pd.concat([stock_data_1,stock_data_2],axis = 0)

In [ ]:
#cut off duplicates
stock_data = stock_data.drop_duplicates(keep='first')

In [ ]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1027022 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1027022 non-null  object        
 1   StockCode    1027022 non-null  object        
 2   Description  1027022 non-null  object        
 3   Quantity     1027022 non-null  int64         
 4   InvoiceDate  1027022 non-null  datetime64[ns]
 5   Price        1027022 non-null  float64       
 6   Customer ID  797815 non-null   float64       
 7   Country      1027022 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 70.5+ MB


In [ ]:
stock_data = stock_data[(stock_data.Country == 'United Kingdom')]

In [ ]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 942334 entries, 0 to 541893
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      942334 non-null  object        
 1   StockCode    942334 non-null  object        
 2   Description  942334 non-null  object        
 3   Quantity     942334 non-null  int64         
 4   InvoiceDate  942334 non-null  datetime64[ns]
 5   Price        942334 non-null  float64       
 6   Customer ID  716069 non-null  float64       
 7   Country      942334 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 64.7+ MB


In [ ]:
stock_data = stock_data.drop(columns = ['Country'], axis = 1)

In [ ]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 942334 entries, 0 to 541893
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      942334 non-null  object        
 1   StockCode    942334 non-null  object        
 2   Description  942334 non-null  object        
 3   Quantity     942334 non-null  int64         
 4   InvoiceDate  942334 non-null  datetime64[ns]
 5   Price        942334 non-null  float64       
 6   Customer ID  716069 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 57.5+ MB


In [ ]:
stock_data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0


In [ ]:
file_name = "/content/drive/My Drive/IEOR4574:Project_1/stock_data.xlsx"
stock_data.to_excel(file_name)

In [ ]:
#Missing values
#Outliers
#Aggregation
